# Requirements

In [1]:
import os
import json
import pandas as pd
import numpy as np
import json
import requests

import re
from itertools import islice # to iterate through dicts

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.oauth2 import service_account # based on google-auth library
import sddk

In [2]:
s = sddk.cloudSession("sciencedata.dk", "SDAM_root", "648597@au.dk")

connection with shared folder established with you as its ordinary user
endpoint variable has been configured to: https://sciencedata.dk/sharingout/648597%40au.dk/SDAM_root/


In [3]:
file_data = json.load(open("../../ServiceAccountsKey.json"))
# (2) transform the content into crendentials object
credentials = service_account.Credentials.from_service_account_info(file_data)
# (3) specify your usage of the credentials
scoped_credentials = credentials.with_scopes(
  ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive'])
# (4) use the constrained credentials for authentication of gspread package
gc = gspread.Client(auth=scoped_credentials)

AGT_overview = gc.open_by_url("https://docs.google.com/spreadsheets/d/1iVta_FuEDgUM_Lf_yByrdbbXNoVH_dnVZs6QRyYv1NM/edit?usp=sharing")
# AGT_metadata = gc.open_by_url("https://docs.google.com/spreadsheets/d/1hEUnL3E07F-EnE3wYnk1V91aXfPDrcnhFHKjD-04CM0/edit?usp=sharing")

In [4]:
LAGT = s.read_file("SDAM_data/AGT/LAGT_dated_20240129.json")

In [5]:
LAGT.head(20)

,author_id,doc_id,filename,author,title,string,wordcount,source,lemmatized_sentences,lemmata_source,tlg_date,not_before,not_after,date_uncertain,tlg_epithet
0,ggm0001,ggm0001.ggm001,ggm0001.ggm001.1st1K-grc1.xml,Anonymous,Anametresis Pontou,1. Χρὴ γινώσκειν ὅτι πάσης τῆς γῆς ὁ περίμετρο...,382,1Kgr,"[[Χρής, γιγνώσκω, γῆ, περίμετρος], [στάδιος, μ...",grecy,None,NaN,NaN,None,[]
2,ogl0001,ogl0001.ogl001,ogl0001.ogl001.1st1K-grc1.xml,Pinytus,De Epistola Pinyti ad Dionysium,"FRAGMENTUM BEATI PINYTI, CNOSSI IN CRETA EPISC...",180,1Kgr,"[[Πινυτός, ἀντιγράφω, θαυμάζω, ἀποδέχω, Διονύσ...",grecy,None,NaN,NaN,None,[]
3,stoa0033a,stoa0033a.tlg028,stoa0033a.tlg028.1st1K-grc1.xml,pseudo-Aristotle,De mundo,ΠΟΛΛΑΚΙΣ μὲν ἔμοιγε θεῖόν τι καὶ δαιμόνιον ὄντ...,6294,1Kgr,"[[θεῖος, δαιμόνιον, χρῆμα, Ἀλέξανδρος, φιλοσοφ...",grecy,None,NaN,NaN,None,[]
4,stoa0033a,stoa0033a.tlg043,stoa0033a.tlg043.1st1K-grc1.xml,pseudo-Aristotle,De spiritu,"ΤΙΣ ἡ τοῦ ἐμφύτου πνεύματος διαμονή, καὶ τίς ἡ...",3459,1Kgr,"[[ἔμφυτος, πνεῦμα, διαμονή, αὔξησις], [Ὁρῶμεν,...",grecy,None,NaN,NaN,None,[]
5,stoa0121,stoa0121.stoa001,stoa0121.stoa001.opp-grc1.xml,Eutropius,Breviarium historiae romanae,Τῆς Ῥωμαϊκῆς βασιλείας ἐν προοιμίοις οὐδὲν ἐγέ...,23867,1Kgr,"[[Ῥωμαϊκός, βασιλεία, προοίμιον, οὐδείς, γίγνο...",grecy,None,NaN,NaN,None,[]
6,stoa0146d,stoa0146d.stoa001,stoa0146d.stoa001.opp-grc1.xml,Hegemonius,Acta Archelai,Μανιχαῖος ἀπόστολος Ἰησοῦ Χριστοῦ καὶ οἱ σὺν ἐ...,5166,1Kgr,"[[Μανιχαῖος, ἀπόστολος, Ἰησοῦς, Χριστός, ἅγιος...",grecy,None,NaN,NaN,None,[]
8,tlg0005,tlg0005.tlg003,tlg0005.tlg003.1st1K-grc1.xml,Theocritus,Syrinx,Οὐδενὸς εὐνάτειρα Μακροπτολέμοιο δὲ μάτηρ μαί...,77,1Kgr,"[[οὐδενός, εὐνητήρ], [μακροπτολέμοιο, μήτηρ, μ...",grecy,4-3 B.C.,-400.0,-201.0,False,[Bucolici]
9,tlg0006,tlg0006.tlg020,tlg0006.tlg020.1st1K-grc1.xml,Euripides,Fragmenta,ποίαν σε φῶμεν γαῖαν ἐκλελοιπότα πόλει ξενοῦσθ...,17708,1Kgr,"[[φημί, γῆ, ἐκλείπω, πόλις, ξενοῦσθαι], [πάτρα...",grecy,5 B.C.,-500.0,-401.0,False,[Tragici]
10,tlg0007,tlg0007.tlg146,tlg0007.tlg146.1st1K-grc1.xml,Plutarch,Παροιμίαι αἷς Ἀλεξανδρεῖς ἐχρῶντο,Οἴκοι τὰ Μιλήσια: ἐπὶ τῶν ὅποι μὴ προςήκει τὴν...,2685,1Kgr,"[[Μιλήσιος], [προςήκω, τρυφή, ἐπιδείκνυμι], [Ἀ...",grecy,A.D. 1-2,1.0,200.0,False,"[Biographi, Philosophici/-ae]"
11,tlg0007,tlg0007.tlg147,tlg0007.tlg147.1st1K-grc1.xml,Plutarch,Ἐκλογὴ περὶ τῶν ἀδυνάτων,Κατὰ πετρῶν σπείρεις. Πλίνθον πλύνεις. Δικτύῳ ...,143,1Kgr,"[[πέτρα, σπείρω], [Πλίνθος, πλύνω, Δίκτυον, ἄν...",grecy,A.D. 1-2,1.0,200.0,False,"[Biographi, Philosophici/-ae]"


# Add our manually coded data for authors (based on tlg metadata from cltk github)

In [6]:
authors_coded = get_as_dataframe(AGT_overview.worksheet("authors_df_CODING"))
authors_coded.head(5)

,author_id,author,author_man,provenience,wikiurl,not_before_man,not_after_man,cent_avg_man,titles,sources,wordcounts
0,ggm0001,Anonymous,NaN,NaN,NaN,NaN,NaN,NaN,['Anametresis Pontou'],['1Kgr'],382.0
1,heb0001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Isaias'],['1Kgr'],40168.0
2,ogl0001,Pinytus,NaN,christian,https://en.wikipedia.org/wiki/Pinytus,101.0,200.0,NaN,['De Epistola Pinyti ad Dionysium'],['1Kgr'],180.0
3,stoa003,pseudo-Aristotle,NaN,pagan,NaN,-300.0,NaN,NaN,"['De mundo', 'De spiritu']","['1Kgr', '1Kgr']",9753.0
4,stoa012,Eutropius,NaN,pagan,https://en.wikipedia.org/wiki/Eutropius_(histo...,363.0,387.0,NaN,['Breviarium historiae romanae'],['1Kgr'],23867.0


In [7]:
authors_coded.set_index("author_id", inplace=True)
notbefore_dict = authors_coded["not_before_man"].to_dict()
notafter_dict = authors_coded["not_after_man"].to_dict()
provenience_dict = authors_coded["provenience"].to_dict()

In [8]:
LAGT["not_before"] = LAGT["author_id"].apply(lambda x: notbefore_dict[x] if x in notbefore_dict.keys() else None)
LAGT["not_after"] = LAGT["author_id"].apply(lambda x: notafter_dict[x] if x in notafter_dict.keys() else None)
LAGT["provenience"] = LAGT["author_id"].apply(lambda x: provenience_dict[x] if x in provenience_dict.keys() else None)

In [9]:
LAGT.sample(10)

,author_id,doc_id,filename,author,title,string,wordcount,source,lemmatized_sentences,lemmata_source,tlg_date,not_before,not_after,date_uncertain,tlg_epithet,provenience
1771,tlg0284,tlg0284.tlg024,tlg0284.tlg024.perseus-grc2.xml,Aelius Aristides,Ἱεροὶ λόγοι δεύτερος,"φέρε δὴ καὶ τῶν ἀνωτέρω μνημονεύσωμεν, ἐάν τι ...",5735,perseus,"[[φέρω, μνημονεύω, τις, δύνω], [ἀρχή, οὐδείς, ...",glaux,A.D. 2,101.0,200.0,False,[Rhetorici],pagan
427,tlg0542,tlg0542.tlg001,tlg0542.tlg001.1st1K-grc1.xml,Julius Pollux,Onomasticon,Τῶν δὲ μέτρων ἔϲτι μέν τι καὶ δάκτυλοϲ· δοχμὴ ...,113,1Kgr,"[[μέτρον, τίς, δάκτυλος], [δοχμή, συγκλείω, δά...",grecy,A.D. 2,101.0,200.0,False,[Grammatici],pagan
985,tlg2200,tlg2200.tlg00551,tlg2200.tlg00551.opp-grc1.xml,Libanius,Declamatio 51,"οἴμοι τοῦ πάθους, οἴμοι τῆς συμφορᾶς. Boiss. p...",1883,1Kgr,"[[πάθος], [συμφορά], [ὑποφέρω], [ψυχή, ἀπερεύξ...",grecy,A.D. 4,301.0,400.0,False,"[Rhetorici, Sophistae]",pagan
1411,tlg0014,tlg0014.tlg001,tlg0014.tlg001.perseus-grc2.xml,Demosthenes,Ὀλυνθιακὸς α΄,"ἀντὶ πολλῶν ἄν, ὦ ἄνδρες Ἀθηναῖοι, χρημάτων ὑμ...",1835,perseus,"[[πολύς, ἀνήρ, Ἀθηναῖος, χρῆμα, αἱρέω, νομίζω,...",glaux,4 B.C.,-400.0,-301.0,False,[Oratores],pagan
475,tlg0591,tlg0591.1st1K002,tlg0591.1st1K002.1st1K-grc1.xml,Antisthenes,Ὀδυσσεύς,"Οὐ πρὸς σέ μοι μόνον ὁ λόγος, δι᾿ ὃν ἀνέστην,...",924,1Kgr,"[[μόνος, λόγος, ἀνίστημι, ἄλλος, ἅπας], [πολύς...",grecy,5-4 B.C.,-500.0,-301.0,False,"[Philosophici/-ae, Rhetorici]",pagan
696,tlg1664,tlg1664.tlg001,tlg1664.tlg001.1st1K-grc1.xml,Seniores Alexandrini,Fragmenta,Ἐπειδή τισιν ἔδοξεν ὁ ψαλμὸς καὶ εἰς Χριστὸν ἀ...,1620,1Kgr,"[[τίς, δοκέω, ψαλμός, Χριστός, ἀναφέρω, λέγω, ...",grecy,A.D. 2,101.0,200.0,False,[Scriptores Ecclesiastici],NaN
1752,tlg0284,tlg0284.tlg005,tlg0284.tlg005.perseus-grc2.xml,Aelius Aristides,Ἡρακλῆς,ἀλλʼ ὦ φίλτατε Ἡράκλεις σέ γε ἐπαινεῖν ἄθλων ὁ...,1485,perseus,"[[φίλος, Ἡρακλέης, ἐπαινέω, ἆθλον, ἡδύς], [πολ...",glaux,A.D. 2,101.0,200.0,False,[Rhetorici],pagan
1052,tlg4019,tlg4019.tlg003,tlg4019.tlg003.1st1K-grc1.xml,Olympiodorus,In Aristotelis meteora commentaria,Πρᾶξις αݲ. p. 338a20 Περὶ μὲν οὖν τῶν πρώτων α...,135382,1Kgr,"[[Πρᾶξις, α], [πρῶτος, αἴτιος, φύσις, προοίμιο...",grecy,A.D. 6,501.0,600.0,False,[Philosophici/-ae],pagan
289,tlg0087,tlg0087.tlg028,tlg0087.tlg028.1st1K-grc1.xml,Herodianus,Περὶ μονοσυλλάβων,πᾶν ὄνομα μονοϲύλλαβον μακροκαταληκτεῖν θέλει\...,280,1Kgr,"[[ὄνομα, μονοσύλλαβος, μακροκαταληκτέω, ἐθέλω,...",grecy,A.D. 2,101.0,200.0,False,"[Grammatici, Rhetorici]",pagan
134,tlg0057,tlg0057.tlg046,tlg0057.tlg046.1st1K-grc1.xml,Galen,De morborum temporibus,Οἷόν τι τοῖς ζώοις ἐστὶν ἡ κατὰ τὰς ἡλικίας δι...,5292,1Kgr,"[[τις, ζῷον, εἰμί, ἡλικία, διαφορά, τοιοῦτος, ...",glaux,A.D. 2,101.0,200.0,False,[Medici],pagan


# Make our corrections on the level of individual documents

In [13]:
paul = ["tlg0031.tlg006", "tlg0031.tlg007", "tlg0031.tlg008", "tlg0031.tlg009", "tlg0031.tlg011", "tlg0031.tlg013", "tlg0031.tlg018"]
pauline = ["tlg0031.tlg010", "tlg0031.tlg012", "tlg0031.tlg014", "tlg0031.tlg015", "tlg0031.tlg016", "tlg0031.tlg017", ]
john = ["tlg0031.tlg004", "tlg0031.tlg023", "tlg0031.tlg024", "tlg0031.tlg025"]
luke = ["tlg0031.tlg003", "tlg0031.tlg005"]

for work in paul:
  LAGT.loc[LAGT["doc_id"]==work, "author_id"] = "tlg0031paul"
  LAGT.loc[LAGT["doc_id"]==work, "author"] = "Paul of Tarsus"

for work in pauline:
  LAGT.loc[LAGT["doc_id"]==work, "author_id"] = "tlg0031pspa"
  LAGT.loc[LAGT["doc_id"]==work, "author"] = "Pauline literature"

for work in john:
  LAGT.loc[LAGT["doc_id"]==work, "author_id"] = "tlg0031john"
  LAGT.loc[LAGT["doc_id"]==work, "author"] = "Johnannine literature"

for work in luke:
  LAGT.loc[LAGT["doc_id"]==work, "author_id"] = "tlg0031luke"
  LAGT.loc[LAGT["doc_id"]==work, "author"] = "Luke (the evangelist)"

In [14]:
import string

for letter, index in zip(list(string.ascii_lowercase)[:14], LAGT[LAGT["author_id"] == "tlg0031"].index.tolist()):
  LAGT.at[index, "author_id"] = "tlg0031" + letter

In [15]:
## updated dates for the longest homeric hymns, based on:
### Faulkner, Andrew, ‘Introduction. Modern Scholarship on the Homeric Hymns: Foundational Issues’, in The Homeric Hymns: Interpretative Essays, ed. by Andrew Faulkner (Oxford: Oxford University Press, 2011), pp. 1–25
LAGT.loc[LAGT["doc_id"].str.startswith("tlg0013"), "author"] = "Homeric hymn"

LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg001", "author_id"] = "tlg0013dyon"
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg001", "not_before"] = -700
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg001", "not_after"] = -601


LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg002", "author_id"] = "tlg0013deme"
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg002", "not_before"] = -700
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg002", "not_after"] = -501


LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg003", "author_id"] = "tlg0013apol"
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg003", "not_before"] = -700
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg003", "not_after"] = -501


LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg004", "author_id"] = "tlg0013herm" # "Ultimately, a late-sixth-century date for Hermes seems most attractive, but the Hymn could also belong to the ﬁrst half of the ﬁfth century."
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg004", "not_before"] = -600
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg004", "not_after"] = -401

LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg005", "author_id"] = "tlg0013aphr"
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg005", "not_before"] = -700
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg005", "not_after"] = -501


LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg007", "author_id"] = "tlg0013dyo2"
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg007", "not_before"] = -600
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg007", "not_after"] = -501

LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg019", "author_id"] = "tlg0013pan"
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg019", "not_before"] = -600
LAGT.loc[LAGT["doc_id"] == "tlg0013.tlg019", "not_after"] = -501


In [16]:
LAGT[LAGT["author_id"].str.startswith("tlg0031")]

,author_id,doc_id,filename,author,title,string,wordcount,source,lemmatized_sentences,lemmata_source,tlg_date,not_before,not_after,date_uncertain,tlg_epithet,provenience
1525,tlg0031a,tlg0031.tlg001,tlg0031.tlg001.perseus-grc2.xml,None,New Testament - Matthew,ΒΙΒΛΟΣ γενέσεως Ἰησοῦ Χριστοῦ υἱοῦ Δαυεὶδ υἱ...,18288,perseus,"[[βίβλος, γένεσις, Ἰησοῦς, Χριστός, υἱός, Δαυί...",morphgnt,A.D. 1,1.0,100.0,False,[],christian
1526,tlg0031b,tlg0031.tlg002,tlg0031.tlg002.perseus-grc2.xml,None,New Testament - Mark,\n\n ΑΡΧΗ τοῦ εὐαγγελίου Ἰη...,11275,perseus,"[[ἀρχή, εὐαγγέλιον, Ἰησοῦς, Χριστός], [γράφω, ...",morphgnt,A.D. 1,1.0,100.0,False,[],christian
1527,tlg0031luke,tlg0031.tlg003,tlg0031.tlg003.perseus-grc2.xml,Luke (the evangelist),New Testament - Luke,\n\n ΕΠΕΙΔΗΠΕΡ ΠΟΛΛΟΙ ἐπεχε...,19459,perseus,"[[πολύς, ἐπιχειρέω, ἀνατάσσομαι, διήγησις, πλη...",morphgnt,A.D. 1,1.0,100.0,False,[],christian
1528,tlg0031john,tlg0031.tlg004,tlg0031.tlg004.perseus-grc2.xml,Johnannine literature,New Testament - John,"\n\n ΕΝ ΑΡΧΗ ἦν ὁ λόγος, κα...",15592,perseus,"[[ἀρχή, εἰμί, λόγος, λόγος, εἰμί, θεός, θεός, ...",morphgnt,A.D. 1,1.0,100.0,False,[],christian
1529,tlg0031luke,tlg0031.tlg005,tlg0031.tlg005.perseus-grc2.xml,Luke (the evangelist),New Testament - Acts,\n τὸν μὲν πρῶτον λόγον ἐποιησάμην περὶ πάντων...,18405,perseus,"[[πρῶτος, λόγος, ποιέω, πᾶς, Θεόφιλος, ἄρχω, Ἰ...",morphgnt,A.D. 1,1.0,100.0,False,[],christian
1530,tlg0031paul,tlg0031.tlg006,tlg0031.tlg006.perseus-grc2.xml,Paul of Tarsus,New Testament - Romans,\n\n ΠΑΥΛΟΣ δοῦλος Ἰησοῦ Χρ...,7107,perseus,"[[Παῦλος, δοῦλος, Χριστός, Ἰησοῦς, κλητός, ἀπό...",morphgnt,A.D. 1,1.0,100.0,False,[],christian
1531,tlg0031paul,tlg0031.tlg007,tlg0031.tlg007.perseus-grc2.xml,Paul of Tarsus,New Testament - 1 Corinthians,\n\n ΠΑΥΛΟΣ κλητὸς ἀπόστολο...,6814,perseus,"[[Παῦλος, κλητός, ἀπόστολος, Χριστός, Ἰησοῦς, ...",morphgnt,A.D. 1,1.0,100.0,False,[],christian
1532,tlg0031paul,tlg0031.tlg008,tlg0031.tlg008.perseus-grc2.xml,Paul of Tarsus,New Testament - 2 Corinthians,\n\n ΠΑΥΛΟΣ ἀπόστολος Χριστ...,4470,perseus,"[[Παῦλος, ἀπόστολος, Χριστός, Ἰησοῦς, θέλημα, ...",morphgnt,A.D. 1,1.0,100.0,False,[],christian
1533,tlg0031paul,tlg0031.tlg009,tlg0031.tlg009.perseus-grc2.xml,Paul of Tarsus,New Testament - Galatians,"\n\n ΠΑΥΛΟΣ ἀπόστολος, οὐκ ...",2235,perseus,"[[Παῦλος, ἀπόστολος, ἄνθρωπος, ἄνθρωπος, Ἰησοῦ...",morphgnt,A.D. 1,1.0,100.0,False,[],christian
1534,tlg0031pspa,tlg0031.tlg010,tlg0031.tlg010.perseus-grc2.xml,Pauline literature,New Testament - Ephesians,\n\n ΠΑΥΛΟΣ ἀπόστολος Χριστ...,2421,perseus,"[[Παῦλος, ἀπόστολος, Χριστός, Ἰησοῦς, θέλημα, ...",morphgnt,A.D. 1,1.0,100.0,False,[],christian


In [17]:
LAGT[LAGT["author_id"].str.startswith("tlg0013")]

,author_id,doc_id,filename,author,title,string,wordcount,source,lemmatized_sentences,lemmata_source,tlg_date,not_before,not_after,date_uncertain,tlg_epithet,provenience
1378,tlg0013dyon,tlg0013.tlg001,tlg0013.tlg001.perseus-grc2.xml,Homeric hymn,Hymn 1 To Dionysus,"\n\nοἳ μὲν γὰρ Δρακάνῳ σʼ, οἳ δʼ Ἰκάρῳ ἠνεμοέσ...",144,perseus,"[[Δράκανος, Ἴκαρος, ἠνεμόεις, φημί, Νάξος, δῖο...",glaux,8-6 B.C.,-700.0,-601.0,False,[],pagan
1379,tlg0013deme,tlg0013.tlg002,tlg0013.tlg002.perseus-grc2.xml,Homeric hymn,Hymn 2 To Demeter,"\nΔήμητρʼ ἠύκομον, σεμνὴν θεόν, ἄρχομʼ ἀείδειν...",3373,perseus,"[[Δήμητηρ, εὔκομος, σεμνός, θεός, ἄρχω, ἀείδω,...",agdt,8-6 B.C.,-700.0,-501.0,False,[],pagan
1380,tlg0013apol,tlg0013.tlg003,tlg0013.tlg003.perseus-grc2.xml,Homeric hymn,Hymn 3 To Apollo,"μνήσομαι οὐδὲ λάθωμαι Ἀπόλλωνος ἑκάτοιο, ὅντε...",3896,perseus,"[[μιμνῄσκω, λανθάνω, Ἀπόλλων, ἕκατος, θεός, δῶ...",glaux,8-6 B.C.,-700.0,-501.0,False,[],pagan
1381,tlg0013herm,tlg0013.tlg004,tlg0013.tlg004.perseus-grc2.xml,Homeric hymn,Hymn 4 To Hermes,"Ἑρμῆν ὕμνει, Μοῦσα, Διὸς καὶ Μαιάδος υἱόν, Κυ...",4031,perseus,"[[Ἑρμῆς, ὑμνέω, Μοῦσα, Ζεύς, Μαιάς, υἱός, Κυλλ...",glaux,8-6 B.C.,-600.0,-401.0,False,[],pagan
1382,tlg0013aphr,tlg0013.tlg005,tlg0013.tlg005.perseus-grc2.xml,Homeric hymn,Hymn 5 To Aphrodite,"μοῦσά μοι ἔννεπε ἔργα πολυχρύσου Ἀφροδίτης, Κ...",2047,perseus,"[[Μοῦσα, ἐνέπω, ἔργον, πολύχρυσος, Ἀφροδίτη, Κ...",glaux,8-6 B.C.,-700.0,-501.0,False,[],pagan
1383,tlg0013,tlg0013.tlg006,tlg0013.tlg006.perseus-grc2.xml,Homeric hymn,Hymn 6 To Aphrodite,"αἰδοίην, χρυσοστέφανον, καλὴν Ἀφροδίτην ᾁσομα...",131,perseus,"[[αἰδοῖος, χρυσοστέφανος, καλός, Ἀφροδίτη, ἀεί...",glaux,8-6 B.C.,-800.0,-601.0,False,[],pagan
1384,tlg0013dyo2,tlg0013.tlg007,tlg0013.tlg007.perseus-grc2.xml,Homeric hymn,Hymn 7 To Dionysus,"ἀμφὶ Διώνυσον, Σεμέλης ἐρικυδέος υἱόν, μνήσομ...",425,perseus,"[[Διόνυσος, Σεμέλη, ἐρικυδής, υἱός, μιμνῄσκω, ...",glaux,8-6 B.C.,-600.0,-501.0,False,[],pagan
1385,tlg0013,tlg0013.tlg008,tlg0013.tlg008.perseus-grc2.xml,Homeric hymn,Hymn 8 To Ares,"Ἆρες ὑπερμενέτα, βρισάρματε, χρυσεοπήληξ, ὀβρ...",100,perseus,"[[Ἄρης, ὑπερμενέτης, βρισάρματε, χρυσεοπήληξ],...",grecy,8-6 B.C.,-800.0,-601.0,False,[],pagan
1386,tlg0013,tlg0013.tlg009,tlg0013.tlg009.perseus-grc2.xml,Homeric hymn,Hymn 9 To Artemis,"Ἄρτεμιν ὕμνει, Μοῦσα, κασιγνήτην Ἑκάτοιο. παρ...",57,perseus,"[[Ἄρτεμις, ὑμνέω, Μοῦσα, κασιγνήτη, Ἑκάτομαι],...",glaux,8-6 B.C.,-800.0,-601.0,False,[],pagan
1387,tlg0013,tlg0013.tlg010,tlg0013.tlg010.perseus-grc2.xml,Homeric hymn,Hymn 10 To Aphrodite,"κυπρογενῆ Κυθέρειαν ἀείσομαι, ἥτε βροτοῖσι με...",39,perseus,"[[Κυπρογενής, Κυθέρεια, ἀείδω, βροτός, μείλιχο...",glaux,8-6 B.C.,-800.0,-601.0,False,[],pagan


In [18]:
len(LAGT)

1818

In [19]:
LAGT["provenience"].notnull().sum()

1660

In [20]:
(LAGT["not_before"].notnull() | LAGT["not_after"].notnull()).sum()

1730

In [21]:
((LAGT["not_before"].notnull() | LAGT["not_after"].notnull()) & (LAGT["provenience"].notnull())).sum()

1652

In [22]:
((LAGT["not_before"] <= 600) | (LAGT["not_after"] <= 600)).sum()

1710

In [23]:
LAGT = LAGT[(LAGT["not_before"] <= 600) | (LAGT["not_after"] <= 600)].copy()
len(LAGT)

1710

In [24]:
LAGT["wordcount"].sum()

32323612

In [25]:
%%time
def ensure_utf8(my_list):
  for sublist in my_list:
    for index, item in enumerate(sublist):
      if isinstance(item, str):
        sublist[index] = item.encode('utf-8', 'replace').decode('utf-8')
  return my_list

LAGT['lemmatized_sentences'] = LAGT['lemmatized_sentences'].apply(ensure_utf8)

CPU times: user 2.8 s, sys: 146 ms, total: 2.95 s
Wall time: 3.02 s


In [26]:
LAGT["lemmatized_sentences"] = LAGT["lemmatized_sentences"].apply(lambda x: [[w for w in s if len(w) > 1] for s in x])
LAGT["lemmatized_sentences"] = LAGT["lemmatized_sentences"].apply(lambda x: [[w for w in s if w[1].islower()] for s in x])

In [27]:
LAGT["lemmatacount"] = LAGT["lemmatized_sentences"].apply(lambda x: len([t for sent in x for t in sent]))
LAGT["lemmatacount"].sum()

13901960

In [29]:
provenience_corrections = {'tlg2038': 'christian',
                             'tlg0572': 'pagan',
                             'tlg2016': 'christian',
                             'tlg2632': 'pagan',
                             'tlg4013': 'pagan',
                             'tlg4016': 'pagan',
                             'tlg4018': 'pagan',
                             'tlg4020': 'pagan',
                             'tlg4021': 'pagan',
                             'tlg4072': 'pagan',
                             'tlg2970': 'pagan',
                             'tlg2233': 'pagan',
                             'tlg4086': 'pagan',
                             'tlg0077': 'pagan',
                             'tlg0580': 'pagan',
                             'tlg0053': 'pagan',
                             'tlg2135': 'pagan',
                             'tlg2136': 'pagan',
                             'tlg2137': 'pagan',
                             'tlg1118': 'pagan',
                             'tlg1126': 'pagan',
                             'tlg1131': 'pagan',
                             'tlg1286': 'pagan',
                             'tlg1367': 'pagan',
                             'tlg1402': 'pagan',
                             'tlg1664': 'pagan'}

In [30]:
for key in provenience_corrections.keys():
    LAGT.loc[LAGT["author_id"] == key, "provenience"] = provenience_corrections[key]

In [31]:
LAGT.loc[LAGT["author_id"] == "tlg2038"]

,author_id,doc_id,filename,author,title,string,wordcount,source,lemmatized_sentences,lemmata_source,tlg_date,not_before,not_after,date_uncertain,tlg_epithet,provenience,lemmatacount
814,tlg2038,tlg2038.tlg001,tlg2038.tlg001.1st1K-grc1.xml,Acta Thomae,Πράξεις τοῦ ἁγίου ἀποστόλου Θωμᾶ,Κατ᾽ ἐκεῖνον τὸν καιρὸν ἦμεν πάντες οἱ ἀπόστολ...,28894,1Kgr,"[[καιρός, ἀπόστολος, Ἰεροσολύμον], [Σίμων, λέγ...",grecy,A.D. 3,201.0,300.0,False,[],christian,13123


In [4]:
LAGT.loc[LAGT["doc_id"] == "tlg2016.tlg001", "not_after"] = 300

In [5]:
LAGT.to_json("../data/large_files/LAGT_v3-0.json")
LAGT.to_parquet("../data/large_files/LAGT_v3-0.parquet")